In [6]:
from os.path import isfile, join
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import numpy as np
import scipy
from bispectrumcode.python.cjh_tests.bispectrum import bispectrum_2d
from analysis.utils.image_bispectrum import *
from analysis.utils.visualization import *
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import cKDTree
%load_ext autoreload
%autoreload 2

/home/sanborn/software/anaconda3/envs/bs/lib/python2.7/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


# Utils

In [7]:
def get_spectra(stim):
    m, n = stim.shape[1], stim.shape[2]
    fft = np.fft.fftshift(np.fft.fft2(stim))
    B = bispectrum_2d(fft, m, n)
    ps = fft * np.conj(fft)
    return B, ps

In [8]:
data_dir = '/home/sanborn/datasets/shapes/'

In [9]:
shapes_neurons = np.load(data_dir+"shapes_neurons.npy")
shapes = np.load(data_dir+"shapes.npy")

In [11]:
B, ps = get_spectra(shapes)

In [17]:
def normalize_rc(rc, responses):
    return np.sum(np.array(rc), axis=0) / np.sum(responses)

def nearest_neighbor(point, set_of_points):
    smallest_distance = np.inf
    for p in set_of_points:
        if np.mean(point - p) ** 2 < smallest_distance:
            closest_point = p
    return closest_point

def rev_corr_nn(neurons, stim, power_spectrum, bispectrum, k=1):
    bs_neighbors = []
    ps_neighbors = []
    pixel_neighbors = []
    for neuron in neurons:
        bs_revcorr = []
        pixel_revcorr = []
        ps_revcorr = []
        responses = []
        for i, response in enumerate(neuron):
            if response > .5 * np.max(neuron):
                ps_revcorr.append(power_spectrum[i] * response)
                bs_revcorr.append(bispectrum[i] * response)
                pixel_revcorr.append(stim[i] * response)
                responses.append(response)
        bs_revcorr = normalize_rc(bs_revcorr, responses)
        ps_revcorr = normalize_rc(ps_revcorr, responses)
        pixel_revcorr = normalize_rc(pixel_revcorr, responses)
        bs_neighbors.append(nearest_neighbor(bs_revcorr, bispectrum))
        pixel_neighbors.append(nearest_neighbor(pixel_revcorr, stim))
        ps_neighbors.append(nearest_neighbor(ps_revcorr, power_spectrum))
    return pixel_neighbors, ps_neighbors, bs_neighbors

In [18]:
pixel_neighbors, ps_neighbors, bs_neighbors = rev_corr_nn([shapes_neurons[0]], shapes, ps, B)

revcorr


In [25]:
plt.imshow(pixel_neighbors[1])

IndexError: list index out of range